In [2]:
!pip install sentence_transformers

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 1.9MB 6.9MB/s 
     |████████████████████████████████| 1.2MB 45.4MB/s 
     |████████████████████████████████| 3.2MB 54.1MB/s 
     |████████████████████████████████| 890kB 51.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp37-none-any.whl size=103068 sha256=0b138b79ccf3d7c3d09df6aa399b5d578e9b04083727c59f9f8db2cbe9ac1114
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=fde2b531d775fc2ead97b5ea3aef02527b7e3575a4d63cf8a6262b03165f172f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Mar  5 20:00:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    22W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [30]:
"""
This examples show how to train a Bi-Encoder for the MS Marco dataset (https://github.com/microsoft/MSMARCO-Passage-Ranking).
The queries and passages are passed independently to the transformer network to produce fixed sized embeddings.
These embeddings can then be compared using cosine-similarity to find matching passages for a given query.
For training, we use MultipleNegativesRankingLoss. There, we pass triplets in the format:
(query, positive_passage, negative_passage)
Negative passage are hard negative examples, that where retrieved by lexical search. We use the negative
passages (the triplets) that are provided by the MS MARCO dataset.
Running this script:
python train_bi-encoder.py
"""
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample
import logging
from datetime import datetime
import gzip
import os
import tarfile
from collections import defaultdict
from torch.utils.data import IterableDataset

import json
from sentence_transformers import SentenceTransformer, util
import time
import torch

from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator


In [6]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout
train_batch_size = 64 #Increasing the train batch size improves the model performance, but requires more GPU memory

# The  model we want to fine-tune


# model_name = 'distilroberta-base'
# word_embedding_model = models.Transformer(model_name, max_seq_length=350)
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# model_name = 'bert-base-uncased'


          

# num_dev_queries = 500           #Number of queries we want to use to evaluate the performance while training
# num_max_dev_negatives = 200     #For every dev query, we use up to 200 hard negatives and add them to the dev corpus

# We construct the SentenceTransformer bi-encoder from scratch


model_name = "/content/drive/MyDrive/SBRT_output/training_biker_bi-encoder-min_5_max_10_ir_10_distilroberta-base_30_iter"
model = SentenceTransformer(model_name)


cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6')

model_save_path = '/content/drive/MyDrive/SBRT_output/training_biker_cross-encoder-30_iter_TinyBERT'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


2021-03-05 19:59:57 - Load pretrained SentenceTransformer: /content/drive/MyDrive/SBRT_output/training_biker_bi-encoder-min_5_max_10_ir_10_distilroberta-base_30_iter
2021-03-05 19:59:57 - Load SentenceTransformer from folder: /content/drive/MyDrive/SBRT_output/training_biker_bi-encoder-min_5_max_10_ir_10_distilroberta-base_30_iter
2021-03-05 20:00:15 - Use pytorch device: cuda
2021-03-05 20:00:16 - Lock 140213675310864 acquired on /root/.cache/huggingface/transformers/7f1fbd2c3307e492c775abdae9d3f1720fb5d02c7c628792823c3f7780b668dd.ef18bb5aedaf4562d772e106aa700d985a4978e6d17f4535445dcf91aff1793a.lock



2021-03-05 20:00:17 - Lock 140213675310864 released on /root/.cache/huggingface/transformers/7f1fbd2c3307e492c775abdae9d3f1720fb5d02c7c628792823c3f7780b668dd.ef18bb5aedaf4562d772e106aa700d985a4978e6d17f4535445dcf91aff1793a.lock
2021-03-05 20:00:17 - Lock 140213674741200 acquired on /root/.cache/huggingface/transformers/b863166bb3a68ffef772c554e4d3dfb63649b224f6bbce230af2440118bd1020.6d4a8837474f85fc8270e78f58a4ed4e1f88b75fadd806aa16aaa486e71eac66.lock



2021-03-05 20:00:24 - Lock 140213674741200 released on /root/.cache/huggingface/transformers/b863166bb3a68ffef772c554e4d3dfb63649b224f6bbce230af2440118bd1020.6d4a8837474f85fc8270e78f58a4ed4e1f88b75fadd806aa16aaa486e71eac66.lock
2021-03-05 20:00:27 - Lock 140213674806992 acquired on /root/.cache/huggingface/transformers/83c5a03c596ad44764c8ac06988f1a689b6c73314511675a8a4e29d9dfea33d2.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock



2021-03-05 20:00:28 - Lock 140213674806992 released on /root/.cache/huggingface/transformers/83c5a03c596ad44764c8ac06988f1a689b6c73314511675a8a4e29d9dfea33d2.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
2021-03-05 20:00:30 - Lock 140213674717008 acquired on /root/.cache/huggingface/transformers/869b6861a5ce65430d510c8cde252813337f19590e0dfb65de9ee3a19ef5a0f6.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock



2021-03-05 20:00:31 - Lock 140213674717008 released on /root/.cache/huggingface/transformers/869b6861a5ce65430d510c8cde252813337f19590e0dfb65de9ee3a19ef5a0f6.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
2021-03-05 20:00:31 - Lock 140213664249360 acquired on /root/.cache/huggingface/transformers/afb6f16d250442d294156f020a8b5e152d602952921a6d11f162a6b349ab0128.21a9fe7063198ccf1c61caed694d678a7974022394d277a08eca70b1cb6c1dd6.lock



2021-03-05 20:00:32 - Lock 140213664249360 released on /root/.cache/huggingface/transformers/afb6f16d250442d294156f020a8b5e152d602952921a6d11f162a6b349ab0128.21a9fe7063198ccf1c61caed694d678a7974022394d277a08eca70b1cb6c1dd6.lock
2021-03-05 20:00:32 - Use pytorch device: cuda


In [ ]:
# model_name = "/content/drive/MyDrive/SBRT_output/training_biker_distilroberta_base_bi-encoder-distilroberta-base-2021-02-21_02-29-11"
# model = SentenceTransformer(model_name)

In [8]:
### Now we read the MS Marco dataset
data_folder = '/content/drive/MyDrive/biker_data/min_5_max_10_ir_10_30k/'
os.makedirs(data_folder, exist_ok=True)
import json 

In [17]:
corpus = {}
collection_filepath = os.path.join(data_folder, 'Corpus_dict.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  corpus = json.load(fIn)

Passage_dict = {}
collection_filepath = os.path.join(data_folder, 'Passage_dict.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  Passage_dict = json.load(fIn)


evaluate_Corpus = {}
collection_filepath = os.path.join(data_folder, 'evaluate_Corpus.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  evaluate_Corpus = json.load(fIn)




def get_triplets(Passage_dict):
    Triplets= []
    for k, v in Passage_dict.items():
        for x in v[0]:
            for y in v[1]:
                # query,same_appi,diff_api
                Triplets.append([k,x,y])
    return Triplets

train_triplets = get_triplets(Passage_dict)


In [12]:
len(train_triplets)

2044710

In [18]:
print(len(corpus))
# print(len(Answers_dict))
print(len(Passage_dict))
print(len(evaluate_queries))
print(len(evaluate_Corpus))
print(len(evaluate_rel_doc))
print(len(dev_queries))
print(len(corpus))
print(len(dev_rel_docs))

21481
21481
1000
20481
1000


In [13]:
train_dataset=[]        
for triplet in train_triplets:
    qid, pos_id, neg_id = triplet
    
    qid = str(qid)
    pos_id = str(pos_id)
    neg_id = str(neg_id)

    query_text = corpus[qid]
    pos_text = corpus[pos_id]
    neg_text = corpus[neg_id]

    pos_instance = InputExample(texts=[query_text, pos_text],label=1)
    neg_instance = InputExample(texts=[query_text, neg_text],label=0)

    train_dataset.append(pos_instance)
    train_dataset.append(neg_instance)

In [19]:
import pandas as pd
df_evaluate = pd.read_csv("/content/drive/MyDrive/biker_data/min_5_max_10_ir_10_30k/cross_encoder_eval.csv")

In [20]:
df_evaluate

,Unnamed: 0,query,passage,is_relevent
0,0,Java using array[] to assign to next index,Want to double the size of the stack when it b...,1
1,1,Java using array[] to assign to next index,copying array by value in java,0
2,2,Java using array[] to assign to next index,How to Store the String Value in Char array in...,0
3,3,Java using array[] to assign to next index,Java: sort one array to another using method a...,0
4,4,Java using array[] to assign to next index,Java - Public String(char[] value),0
...,...,...,...,...
8682,8682,issue with date and time zone how to get date ...,Formatting currency digits with commas,0
8683,8683,issue with date and time zone how to get date ...,Are there constants for language codes in java...,0
8684,8684,issue with date and time zone how to get date ...,Java 8 does not render currency symbol,0
8685,8685,issue with date and time zone how to get date ...,How to set customize currency in java?,0


In [25]:
dev_samples = []
for index, row in df_evaluate.iterrows():
    query_text = row["query"]
    passage_text = row["passage"]
    label = int(row["is_relevent"])
    dev_samples.append(InputExample(texts = [query_text, passage_text], label = label))

In [27]:
len(dev_samples)

8687

In [33]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

evaluator = CEBinaryClassificationEvaluator.from_input_examples(dev_samples, name='cross_encoder')

In [35]:
# Train the model
cross_encoder.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=20,
          evaluation_steps=5000,
          warmup_steps=1000,
          save_best_model=True,
          output_path=model_save_path)

2021-03-05 20:24:34 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 0 after 5000 steps:
2021-03-05 20:24:38 - Accuracy:           88.50	(Threshold: 0.9992)
2021-03-05 20:24:38 - F1:                 25.21	(Threshold: 0.9173)
2021-03-05 20:24:38 - Precision:          19.86
2021-03-05 20:24:38 - Recall:             34.50
2021-03-05 20:24:38 - Average Precision:  18.09

2021-03-05 20:24:38 - Save model to /content/drive/MyDrive/SBRT_output/training_biker_cross-encoder-30_iter_TinyBERT-2021-03-05_20-00-32
2021-03-05 20:33:52 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 0 after 10000 steps:
2021-03-05 20:33:56 - Accuracy:           88.48	(Threshold: 0.9992)
2021-03-05 20:33:56 - F1:                 24.68	(Threshold: 0.8793)
2021-03-05 20:33:56 - Precision:          15.26
2021-03-05 20:33:56 - Recall:             64.40
2021-03-05 20:33:56 - Average Precision:  17.97

2021-03-05 20:43:14 - CEBinaryClassifica

2021-03-05 22:23:47 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 1 after 5000 steps:
2021-03-05 22:23:50 - Accuracy:           88.48	(Threshold: 0.9999)
2021-03-05 22:23:51 - F1:                 27.44	(Threshold: 0.9907)
2021-03-05 22:23:51 - Precision:          16.81
2021-03-05 22:23:51 - Recall:             74.60
2021-03-05 22:23:51 - Average Precision:  20.06

2021-03-05 22:23:51 - Save model to /content/drive/MyDrive/SBRT_output/training_biker_cross-encoder-30_iter_TinyBERT-2021-03-05_20-00-32
2021-03-05 22:33:09 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 1 after 10000 steps:
2021-03-05 22:33:13 - Accuracy:           88.51	(Threshold: 0.9999)
2021-03-05 22:33:13 - F1:                 26.86	(Threshold: 0.9922)
2021-03-05 22:33:13 - Precision:          16.43
2021-03-05 22:33:13 - Recall:             73.60
2021-03-05 22:33:13 - Average Precision:  19.34

2021-03-05 22:42:31 - CEBinaryClassifica

2021-03-06 00:22:48 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 2 after 5000 steps:
2021-03-06 00:22:52 - Accuracy:           88.49	(Threshold: 1.0000)
2021-03-06 00:22:52 - F1:                 28.34	(Threshold: 0.9996)
2021-03-06 00:22:52 - Precision:          18.09
2021-03-06 00:22:52 - Recall:             65.40
2021-03-06 00:22:52 - Average Precision:  19.97

2021-03-06 00:32:07 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 2 after 10000 steps:
2021-03-06 00:32:10 - Accuracy:           88.48	(Threshold: 1.0000)
2021-03-06 00:32:10 - F1:                 28.50	(Threshold: 0.9999)
2021-03-06 00:32:10 - Precision:          18.29
2021-03-06 00:32:10 - Recall:             64.50
2021-03-06 00:32:10 - Average Precision:  20.29

2021-03-06 00:41:24 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 2 after 15000 steps:
2021-03-06 00:41:28 - Accuracy:           88.4

2021-03-06 02:19:58 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 3 after 5000 steps:
2021-03-06 02:20:02 - Accuracy:           88.49	(Threshold: 1.0000)
2021-03-06 02:20:02 - F1:                 29.41	(Threshold: 1.0000)
2021-03-06 02:20:02 - Precision:          18.41
2021-03-06 02:20:02 - Recall:             73.00
2021-03-06 02:20:02 - Average Precision:  20.16

2021-03-06 02:29:04 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 3 after 10000 steps:
2021-03-06 02:29:07 - Accuracy:           88.48	(Threshold: 1.0000)
2021-03-06 02:29:07 - F1:                 30.16	(Threshold: 1.0000)
2021-03-06 02:29:07 - Precision:          19.24
2021-03-06 02:29:07 - Recall:             69.80
2021-03-06 02:29:07 - Average Precision:  21.58

2021-03-06 02:38:10 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 3 after 15000 steps:
2021-03-06 02:38:14 - Accuracy:           88.4

2021-03-06 04:16:21 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 4 after 5000 steps:
2021-03-06 04:16:25 - Accuracy:           88.50	(Threshold: 1.0000)
2021-03-06 04:16:25 - F1:                 29.93	(Threshold: 1.0000)
2021-03-06 04:16:25 - Precision:          19.74
2021-03-06 04:16:25 - Recall:             61.80
2021-03-06 04:16:25 - Average Precision:  21.28

2021-03-06 04:25:19 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 4 after 10000 steps:
2021-03-06 04:25:23 - Accuracy:           88.49	(Threshold: 1.0000)
2021-03-06 04:25:23 - F1:                 30.09	(Threshold: 1.0000)
2021-03-06 04:25:23 - Precision:          20.71
2021-03-06 04:25:23 - Recall:             55.00
2021-03-06 04:25:23 - Average Precision:  22.28

2021-03-06 04:34:19 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 4 after 15000 steps:
2021-03-06 04:34:23 - Accuracy:           88.4

...

2021-03-06 08:05:19 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 6 after 5000 steps:
2021-03-06 08:05:23 - Accuracy:           88.50	(Threshold: 1.0000)
2021-03-06 08:05:23 - F1:                 31.70	(Threshold: 1.0000)
2021-03-06 08:05:23 - Precision:          21.79
2021-03-06 08:05:23 - Recall:             58.10
2021-03-06 08:05:23 - Average Precision:  22.72

2021-03-06 08:14:15 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 6 after 10000 steps:
2021-03-06 08:14:19 - Accuracy:           88.50	(Threshold: 1.0000)
2021-03-06 08:14:19 - F1:                 32.13	(Threshold: 1.0000)
2021-03-06 08:14:19 - Precision:          20.56
2021-03-06 08:14:19 - Recall:             73.50
2021-03-06 08:14:19 - Average Precision:  23.06

2021-03-06 08:23:15 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 6 after 15000 steps:
2021-03-06 08:23:19 - Accuracy:           88.4

2021-03-06 10:00:59 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 7 after 5000 steps:
2021-03-06 10:01:03 - Accuracy:           88.49	(Threshold: 1.0000)
2021-03-06 10:01:03 - F1:                 31.22	(Threshold: 1.0000)
2021-03-06 10:01:03 - Precision:          20.13
2021-03-06 10:01:03 - Recall:             69.60
2021-03-06 10:01:03 - Average Precision:  20.53

2021-03-06 10:10:05 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 7 after 10000 steps:
2021-03-06 10:10:08 - Accuracy:           88.48	(Threshold: 1.0000)
2021-03-06 10:10:08 - F1:                 31.15	(Threshold: 1.0000)
2021-03-06 10:10:08 - Precision:          19.50
2021-03-06 10:10:08 - Recall:             77.40
2021-03-06 10:10:08 - Average Precision:  19.72

2021-03-06 10:19:11 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 7 after 15000 steps:
2021-03-06 10:19:15 - Accuracy:           88.5

2021-03-06 11:57:36 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 8 after 5000 steps:
2021-03-06 11:57:40 - Accuracy:           88.49	(Threshold: 1.0000)
2021-03-06 11:57:40 - F1:                 32.08	(Threshold: 1.0000)
2021-03-06 11:57:40 - Precision:          21.11
2021-03-06 11:57:40 - Recall:             66.70
2021-03-06 11:57:40 - Average Precision:  21.64

2021-03-06 12:06:46 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 8 after 10000 steps:
2021-03-06 12:06:49 - Accuracy:           88.48	(Threshold: 1.0000)
2021-03-06 12:06:49 - F1:                 31.67	(Threshold: 1.0000)
2021-03-06 12:06:49 - Precision:          20.28
2021-03-06 12:06:49 - Recall:             72.20
2021-03-06 12:06:49 - Average Precision:  21.97

2021-03-06 12:15:52 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 8 after 15000 steps:
2021-03-06 12:15:56 - Accuracy:           88.5

2021-03-06 13:54:26 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 9 after 5000 steps:
2021-03-06 13:54:30 - Accuracy:           88.50	(Threshold: 1.0000)
2021-03-06 13:54:30 - F1:                 32.12	(Threshold: 1.0000)
2021-03-06 13:54:30 - Precision:          21.65
2021-03-06 13:54:30 - Recall:             62.20
2021-03-06 13:54:30 - Average Precision:  22.55

2021-03-06 14:03:37 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 9 after 10000 steps:
2021-03-06 14:03:42 - Accuracy:           88.48	(Threshold: 1.0000)
2021-03-06 14:03:42 - F1:                 32.56	(Threshold: 1.0000)
2021-03-06 14:03:42 - Precision:          21.04
2021-03-06 14:03:42 - Recall:             72.00
2021-03-06 14:03:42 - Average Precision:  21.06

2021-03-06 14:12:48 - CEBinaryClassificationEvaluator: Evaluating the model on cross_encoder dataset in epoch 9 after 15000 steps:
2021-03-06 14:12:52 - Accuracy:           88.5

KeyboardInterrupt: ignored

In [ ]:
cross_encoder.evaluate(evaluator=evaluator)

2021-03-03 04:42:32 - Information Retrieval Evaluation on distilroberta-train_eval dataset:
2021-03-03 04:42:42 - Accuracy@1: 0.00%
2021-03-03 04:42:42 - Accuracy@3: 58.30%
2021-03-03 04:42:42 - Accuracy@5: 68.80%
2021-03-03 04:42:42 - Accuracy@10: 80.70%
2021-03-03 04:42:42 - Precision@1: 0.00%
2021-03-03 04:42:42 - Precision@3: 30.07%
2021-03-03 04:42:42 - Precision@5: 34.60%
2021-03-03 04:42:42 - Precision@10: 37.60%
2021-03-03 04:42:42 - Recall@1: 0.00%
2021-03-03 04:42:42 - Recall@3: 2.00%
2021-03-03 04:42:42 - Recall@5: 3.85%
2021-03-03 04:42:42 - Recall@10: 7.52%
2021-03-03 04:42:42 - MRR@10: 0.3102
2021-03-03 04:42:42 - NDCG@10: 0.3341
2021-03-03 04:42:42 - MAP@100: 0.2899
2021-03-03 04:42:42 - Queries: 1000
2021-03-03 04:42:42 - Corpus: 21481



0.28987622595921436

In [ ]:
# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('msmarco-distilbert-base-v2')

dev_q = list(dev_queries.items())[:10]
# dev_r = list(dev_rel_docs.items())[:10]

for (k,v) in dev_q:
  query = v
  passage_id = list(dev_rel_docs[k])[0]
  passage = dev_corpus[passage_id]
  print(query)
  print(passage_id)
  print(passage)
  query_embedding = model.encode(query)
  passage_embedding = model.encode(passage)

  print("Similarity:", util.pytorch_cos_sim(query_embedding, passage_embedding))